In [2]:
library(Seurat)

# Download data (tcell rds and metadata) from
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE123813

count_file <- "test_data/scrna_metabolic/GSE123813_bcc_scRNA_counts.txt.gz"
meta_file <- "test_data/scrna_metabolic/GSE123813_bcc_all_metadata.txt.gz"

counts <- read.table(count_file, header = TRUE, row.names = 1, sep = "\t", check.names = F)
metadata <- read.table(meta_file, header = TRUE, row.names = 1, sep = "\t", check.names = F)

In [8]:
# Subset 1000 cells for jusb demo purpose
counts = counts[, sample(1:ncol(counts), 1000)]
metadata = metadata[colnames(counts),]

In [21]:
# Create seurat object
seurat_obj = CreateSeuratObject(counts=counts)
seurat_obj@meta.data = cbind(
    seurat_obj@meta.data, 
    metadata[rownames(seurat_obj@meta.data),]
)
seurat_obj = NormalizeData(seurat_obj)
all.genes <- rownames(seurat_obj)
seurat_obj <- ScaleData(seurat_obj, features = all.genes)
seurat_obj <- FindVariableFeatures(object = seurat_obj)
seurat_obj <- RunPCA(seurat_obj, features = VariableFeatures(object = seurat_obj))

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Centering and scaling data matrix

PC_ 1 
Positive:  CALD1, EMP2, SPARC, CAV1, EMP1, ACTN1, KRT17, BGN, DSP, RAB13 
	   RND3, S100A16, KRT14, S100A14, S100A2, CD9, FHL2, IER3, GJB2, TRIM29 
	   TSC22D1, KRT15, SFN, FSTL1, DDR1, IGFBP7, JUP, PTRF, KRT5, FOSL1 
Negative:  CD74, CRIP1, RARRES3, RGS1, LAT, CD69, NKG7, HLA-DPA1, TNFRSF4, CD27 
	   GZMA, HLA-DRB1, CXCR6, CTSW, GPR183, LDLRAD4, ICOS, HIST1H4C, GZMK, AC092580.4 
	   SLC9A3R1, CCR7, S100A4, HLA-DPB1, HMGB2, GBP5, SELL, CXCR3, LAG3, HLA-DRB5 
PC_ 2 
Positive:  DSP, DSC3, SFN, SERPINB5, KRT17, S100A14, KRT15, KRT16, IRF6, TRIM29 
	   KRT14, LGALS7B, JUP, PERP, TACSTD2, GJB2, KRT5, KRT6B, DDR1, S100A2 
	   PKP1, CDH3, KRT6A, FXYD3, GJB3, MPZL2, CXADR, DSC2, DSG3, GRHL3 
Negative:  COL1A2, COL3A1, LUM, COL6A1, MXRA8, FN1, CTSK, COL1A1, COL6A3, DCN 
	   MMP2, COL6A2, PRRX1, FKBP10, TNFAIP6, FAP, PCOLCE, PDGFRB, NNMT, AEBP1 
	   C1S, CCDC80, S

In [22]:
# Cluster the cells
# Or you can ask the pipeline to do it by setting 
# `pipeline.scrna_metabolic.clustered` to `FALSE` in your
# `~/.biopipen.toml`

seurat_obj <- FindNeighbors(seurat_obj, dims = 1:10)
seurat_obj <- FindClusters(seurat_obj, resolution = 0.5)
head(Idents(seurat_obj))

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 1000
Number of edges: 30631

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8795
Number of communities: 9
Elapsed time: 0 seconds


bcc.su009.post.tcell_CCATTCGCAATCTACG  bcc.su004.pre.tcell_ACAGCTACACTTCTGC 
                                    0                                     0 
 bcc.su006.pre.tcell_CGTGTAAAGTGACTCT bcc.su001.post.tcell_CCTTTCTGTACCGTTA 
                                    1                                     2 
 bcc.su007.pre.tcell_ATTTCTGAGAAGGACA  bcc.su009.pre.tcell_AGACGTTTCCTGCAGG 
                                    1                                     8 
Levels: 0 1 2 3 4 5 6 7 8

In [23]:
head(seurat_obj@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,patient,treatment,sort,cluster,UMAP1,UMAP2,RNA_snn_res.0.5,seurat_clusters
,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<fct>,<fct>
bcc.su009.post.tcell_CCATTCGCAATCTACG,bcc.su009.post.tcell,4242,1726,su009,post,CD45+ CD3+,CD8_mem_T_cells,-9.1816435,0.7484789,0,0
bcc.su004.pre.tcell_ACAGCTACACTTCTGC,bcc.su004.pre.tcell,3159,1466,su004,pre,CD45+ CD3+,CD8_ex_T_cells,4.1067562,3.3754938,0,0
bcc.su006.pre.tcell_CGTGTAAAGTGACTCT,bcc.su006.pre.tcell,3279,1401,su006,pre,CD45+ CD3+,Tregs,0.4816457,11.9388428,1,1
bcc.su001.post.tcell_CCTTTCTGTACCGTTA,bcc.su001.post.tcell,5057,2218,su001,post,CD45+ CD3+,CD8_ex_T_cells,2.3045983,7.4856248,2,2
bcc.su007.pre.tcell_ATTTCTGAGAAGGACA,bcc.su007.pre.tcell,3701,1413,su007,pre,CD45+ CD3+,CD8_mem_T_cells,-1.9617293,5.5546365,1,1
bcc.su009.pre.tcell_AGACGTTTCCTGCAGG,bcc.su009.pre.tcell,3891,1593,su009,pre,CD45+ CD3+,Tcell_prolif,5.2243228,-1.0460106,8,8


In [24]:
saveRDS(seurat_obj, "test_data/scrna_metabolic/seurat_obj.rds")

In [25]:
table(seurat_obj@meta.data$patient)


su001 su002 su003 su004 su005 su006 su007 su008 su009 su010 su012 
  204     4    23    23   137   178    43   145   165    13    65 